In [6]:
import numpy as np


In [7]:




%load_ext autoreload

import time
import json
import os
import numpy as np
import flwr as fl
import pickle
from math import floor

from hydra import initialize, compose
from omegaconf import OmegaConf, DictConfig

from logging import INFO, DEBUG
from flwr.common.logger import log



with initialize(version_base=None, config_path="./config/"):
    cfg = compose(config_name='config.yaml')
    print(OmegaConf.to_yaml(cfg))


from src.data.dataset_info import datasets
data_root = "./datasets/partitions/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
multi_class: false
n_clients: 10
n_rounds: 20
config_fit:
  momentum: 0.9
  local_epochs: 3
  batch_size: 128



In [9]:
NUM_CLIENTS = 10
BATCH_SIZE = 128
folder_path = "./datasets/preprocessed/"
clients_paths = [
    folder_path + "client_0.pkl",
    folder_path + "client_1.pkl",
    folder_path + "client_2.pkl",
    folder_path + "client_3.pkl",
    folder_path + "client_4.pkl",
    folder_path + "client_5.pkl",
    folder_path + "client_6.pkl",
    folder_path + "client_7.pkl",
    folder_path + "client_8.pkl",
    folder_path + "client_9.pkl",
]



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


def read_clients(folder_path, clients_paths, label_col, class_col, class_num_col, drop_columns, weak_columns):
    test = pd.read_pickle(folder_path + "test.pkl")

    if multi_class:
        test[label_col] = test[class_num_col]


   # ""test_by_class = {}
   # classes = test[class_col].unique()
   # for class_value in classes:
   #     test_class = test[test[class_col] == class_value].copy()
   #     test_class.drop(drop_columns, axis=1, inplace=True)
   #     test_class.drop(weak_columns, axis=1, inplace=True)
   #     test_class.reset_index(drop=True, inplace=True)

   #    test_class_labels = test_class[label_col].to_numpy()
   #    test_class = test_class.drop([label_col], axis=1).to_numpy()

   #    test_by_class[class_value] = (test_class, test_class_labels)""

    test.drop(drop_columns, axis=1, inplace=True)
    test.drop(weak_columns, axis=1, inplace=True)
    test.reset_index(drop=True, inplace=True)

    test_labels = test[label_col].to_numpy()
    test = test.drop([label_col], axis=1).to_numpy()
    input_dim = test.shape[1]

    client_data = []
    for client_path in clients_paths:
        client_data.append(pd.read_pickle(client_path))

    for i in range(len(client_data)):

        cdata = client_data[i]

        if multi_class:
            cdata[label_col] = cdata[class_num_col]
       

        cdata.drop(drop_columns, axis=1, inplace=True)
        cdata.drop(weak_columns, axis=1, inplace=True)
        cdata.reset_index(drop=True, inplace=True)

        # Split into train, validation, and test sets
        c_train, c_test = train_test_split(cdata, test_size=0.1)

        # Split c_train further into c_train and c_val
        c_train, c_val = train_test_split(c_train, test_size=0.2)

        # Extract labels and features for train, validation, and test
        y_train = c_train[label_col].to_numpy()
        x_train = c_train.drop([label_col], axis=1).to_numpy()

        y_val = c_val[label_col].to_numpy()
        x_val = c_val.drop([label_col], axis=1).to_numpy()

        y_test = c_test[label_col].to_numpy()
        x_test = c_test.drop([label_col], axis=1).to_numpy()

        # Store in client_data: (x_train, y_train, x_val, y_val, x_test, y_test)
        client_data[i] = (x_train, y_train, x_val, y_val, x_test, y_test)

    return client_data, test, test_labels, test_by_class, input_dim

In [2]:
# specifying the number of classes, since it is different from one dataset to another and also if binary or multi-class classification
classes_set = {"benign", "attack"}
labels_names = {0: "benign", 1: "attack"}
num_classes = 2
if cfg.multi_class:
    classes_set = {"BENIGN", "SSH-Patator", "FTP-Patator", "DoS slowloris", "Heartbleed", "DoS Slowhttptest", "DoS Hulk", "DoS GoldenEye", "Infiltration", "Web Attack � Brute Force", "Web Attack � XSS", "Web Attack � Sql Injection", "PortScan", "DDoS", "Bot" }
    labels_names = {0: 'BENIGN', 1: 'Bot', 2: 'DDoS', 3: 'DoS GoldenEye', 4: 'DoS Hulk', 5: 'DoS Slowhttptest', 6: 'DoS slowloris', 7: 'FTP-Patator', 8: 'Heartbleed', 9: 'Infiltration', 10: 'PortScan', 11: 'SSH-Patator', 12: 'Web Attack � Brute Force', 13: 'Web Attack � Sql Injection', 14: 'Web Attack � XSS'}
    num_classes = len(classes_set)
    
labels_names = {int(k): v for k, v in labels_names.items()}

print(f"==>> classes_set: {classes_set}")
print(f"==>> num_classes: {num_classes}")
print(f"==>> labels_names: {labels_names}")

==>> classes_set: {'Web Attack � XSS', 'DoS Hulk', 'Heartbleed', 'DoS Slowhttptest', 'Web Attack � Brute Force', 'Web Attack � Sql Injection', 'SSH-Patator', 'Infiltration', 'BENIGN', 'DoS GoldenEye', 'DDoS', 'DoS slowloris', 'FTP-Patator', 'Bot', 'PortScan'}
==>> num_classes: 15
==>> labels_names: {0: 'BENIGN', 1: 'Bot', 2: 'DDoS', 3: 'DoS GoldenEye', 4: 'DoS Hulk', 5: 'DoS Slowhttptest', 6: 'DoS slowloris', 7: 'FTP-Patator', 8: 'Heartbleed', 9: 'Infiltration', 10: 'PortScan', 11: 'SSH-Patator', 12: 'Web Attack � Brute Force', 13: 'Web Attack � Sql Injection', 14: 'Web Attack � XSS'}


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

def create_nids_model(input_shape, num_classes):
    model = Sequential(name='NIDS_CNN')
    
    # Feature extraction block 1
    model.add(Conv1D(64, kernel_size=3, activation='relu', 
                     input_shape=input_shape, padding='same',
                     kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    
    # Feature extraction block 2
    model.add(Conv1D(128, kernel_size=3, activation='relu', 
                     padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.4))
    
    # Feature extraction block 3
    model.add(Conv1D(256, kernel_size=2, activation='relu', 
                     padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    
    # Classification block
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

# Example usage:
# model = create_nids_model(input_shape=(41, 1), num_classes=5)
# model.summary()

In [11]:
results_final = {}


In [12]:
class FlClient(fl.client.NumPyClient):
    def __init__(self, context, model, x_train, y_train, x_val, y_val, x_test, y_test, input_dim):

        self.context = context
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val  
        self.x_test, self.y_test = x_test, y_test

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters):
        self.model.set_weights(parameters)
        
    def fit(self, parameters, config):

        lr=0.001
        self.model = create_keras_model(input_shape=self.input_dim, alpha=lr)
        self.set_parameters(parameters, config)

        history = self.model.fit(self.x_train,self.y_train
                                validation_data=(self.x_val, self.y_val),  
                                epochs=config["local_epochs"],
                                batch_size=config["batch_size"],
                                verbose=0)

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.X_val, self.y_val, 2, verbose=0)
        return loss, len(self.X_val), {"accuracy": accuracy}

In [13]:
def generate_client_fn(data, labels):

    def client_fn(context: Context):

        client_id = int(context.node_config["partition-id"])
        print(f"==>> client_id: {client_id}")

        X_train, X_val, y_train, y_val = train_test_split(
            data[client_id], labels[client_id], test_size=0.1, random_state=1)

        print(f"==>> X_train: {X_train}")
        print(f"==>> X_val: {X_val}")
        
        
        model = create_keras_model(6)

        return FLClient(
            context, np.array(X_train), np.array(X_val), np.array(y_train), np.array(y_val), model, 1
        ).to_client()

    return client_fn


NameError: name 'Context' is not defined

In [ ]:
# Create the ClientApp
client_app = ClientApp(client_fn=generate_client_fn(
    clients_data, clients_labels))

In [ ]:
def get_evaluate_fn(x_test_server, y_test_server):
    def evaluate_fn(server_round: int, parameters, config):
        eval_model = create_keras_model(input_shape=6)
        eval_model.set_weights(parameters)

        test_loss, test_acc = eval_model.evaluate(x_test_server, y_test_server,
                                                  batch_size = 2)
        
        results_dict = {
            "test_loss": test_loss,
            "test_acc": test_acc,
            "round": server_round
        }

        return results_dict

    return evaluate_fn


In [ ]:
def get_on_fit_config():

    def fit_config_fn(server_round: int):
        return {
            "lr": 0.001,
            "local_epochs": 1,
            "batch_size": 2,
        }

    return fit_config_fn

In [ ]:
def generate_server_fn(data, labels):
    def server_fn(context: Context):
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
            min_fit_clients=len(clients_data),  # number of clients to sample for fit()
            fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
            min_evaluate_clients=0,  # number of clients to sample for evaluate()
            min_available_clients=len(clients_data),
            on_fit_config_fn=get_on_fit_config(),  # a function to execute to obtain the configuration to send to the clients during fit()
            evaluate_fn=get_evaluate_fn(data, labels),
        )
        
        return ServerAppComponents(
            strategy=strategy,
            config=ServerConfig(num_rounds=2)
        )

    return server_fn
server_app = ServerApp(server_fn=generate_server_fn(data, labels))


In [ ]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [ ]:
# Create FedAvg strategy
strategy = FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=10,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=10,  # Wait until all 10 clients are available
)

In [ ]:
client_data, test, test_labels, test_by_class, input_dim = read_clients(
    folder_path, clients_paths, dataset.label_col, dataset.class_col, dataset.class_num_col, centralities_columns, pca_columns, dataset.drop_columns, dataset.weak_columns, cfg.multi_class)

In [ ]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)